In [43]:
!pip install yfinance google-cloud-storage pandas numpy matplotlib seaborn tensorflow kaleido google-cloud-bigquery

In [44]:
import pandas as pd
import numpy as np
from google.cloud import storage
import io
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.graph_objects as go
from google.cloud import bigquery

In [45]:
# ================================================
# Configurações e Funções de Carregamento
# ================================================
tickers = [
    'AMBP3.SA', 'LIGT3.SA', 'EMBR3.SA', 'SYNE3.SA', 'MRFG3.SA', 'BRFS3.SA',
    'JBSS3.SA', 'CVCB3.SA', 'PETR4.SA', 'VALE3.SA', 'ELET3.SA', 'BBDC4.SA',
    'BBAS3.SA', 'ITUB4.SA', 'ABEV3.SA', 'MGLU3.SA', 'WEGE3.SA', 'CMIG4.SA',
    '^BVSP'
]

bucket_name = "fiap-augusto"  # Ajuste para o seu bucket GCS
dataset_id = "financeiro"     # Ajuste para o seu dataset no BigQuery

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

def load_data_from_gcs(ticker):
    file_name = f"{ticker.replace('^','_')}.csv"
    blob = bucket.blob(f"tickers/{ticker}/{file_name}")
    data_str = blob.download_as_string()
    df = pd.read_csv(
        io.BytesIO(data_str),
        skiprows=3,  # Ignora as 3 primeiras linhas do arquivo
        header=None, # Não usar linha de cabeçalho
        names=["Date","Price","Adj Close","Close","High","Low","Open","Volume"],
        index_col="Date",
        parse_dates=["Date"]
    )
    return df

data_dict = {t: load_data_from_gcs(t) for t in tickers}

prices = pd.DataFrame({t: data_dict[t]['Adj Close'] for t in tickers})
prices = prices.dropna()
returns = prices.pct_change().dropna()


In [46]:
  # Remover o índice de mercado da lista de colunas para a seleção de carteiras
returns_no_market = returns.drop(columns='^BVSP', errors='ignore')

# ================================================
# Construção das Carteiras (sem ^BVSP)
# ================================================
cumulative_returns = (1+returns_no_market).prod() - 1
top_assets = cumulative_returns.sort_values(ascending=False).index[:5]

vol = returns_no_market.std()
low_vol_assets = vol.sort_values().index[:5]

corr_matrix = returns_no_market.corr()
mean_corr = corr_matrix.mean(axis=1)
low_corr_assets = mean_corr.sort_values().index[:5]

carteira_alta_rentabilidade = returns[top_assets].mean(axis=1)
carteira_baixa_vol = returns[low_vol_assets].mean(axis=1)
carteira_baixa_corr = returns[low_corr_assets].mean(axis=1)

# Índice de mercado permanece o mesmo
mercado = returns['^BVSP']

  def train_lstm(series, look_back=30):
      data = series.values
      X, y = [], []
      forecast_horizon = 1
      for i in range(len(data)-look_back-forecast_horizon):
          X.append(data[i:i+look_back])
          y.append(data[i+look_back])
      X = np.array(X)
      y = np.array(y)
      X = X.reshape(X.shape[0], X.shape[1], 1)

      model = Sequential()
      model.add(LSTM(50, input_shape=(look_back,1)))
      model.add(Dense(1))
      model.compile(loss='mse', optimizer='adam')
      model.fit(X, y, epochs=10, batch_size=32, verbose=0)
      return model

  def forecast_future(model, series, look_back=30, steps=60):
      data = series.values
      forecast = []
      last_input = data[-look_back:]
      for _ in range(steps):
          X = last_input.reshape(1, look_back, 1)
          pred = model.predict(X, verbose=0)[0][0]
          forecast.append(pred)
          last_input = np.append(last_input[1:], pred)
      return pd.Series(forecast, index=pd.date_range(start=series.index[-1]+pd.Timedelta(days=1), periods=steps, freq='D'))

  # Treinando modelos e fazendo previsões
  model_alta = train_lstm(carteira_alta_rentabilidade)
  model_baixa_vol = train_lstm(carteira_baixa_vol)
  model_baixa_corr = train_lstm(carteira_baixa_corr)

  pred_alta_series = forecast_future(model_alta, carteira_alta_rentabilidade, steps=60)
  pred_baixa_vol_series = forecast_future(model_baixa_vol, carteira_baixa_vol, steps=60)
  pred_baixa_corr_series = forecast_future(model_baixa_corr, carteira_baixa_corr, steps=60)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [47]:
# ================================================
# DataFrames Histórico e Previsão
# ================================================
comparison_df = pd.DataFrame({
    'Alta_Rentabilidade': (1+carteira_alta_rentabilidade).cumprod(),
    'Baixa_Vol': (1+carteira_baixa_vol).cumprod(),
    'Baixa_Corr': (1+carteira_baixa_corr).cumprod(),
    'Mercado': (1+mercado).cumprod()
}, index=carteira_alta_rentabilidade.index)

last_alta = comparison_df['Alta_Rentabilidade'].iloc[-1]
last_baixa_vol = comparison_df['Baixa_Vol'].iloc[-1]
last_baixa_corr = comparison_df['Baixa_Corr'].iloc[-1]

alta_pred_cum = last_alta * (1 + pred_alta_series).cumprod()
baixa_vol_pred_cum = last_baixa_vol * (1 + pred_baixa_vol_series).cumprod()
baixa_corr_pred_cum = last_baixa_corr * (1 + pred_baixa_corr_series).cumprod()

predictions_df = pd.DataFrame({
    'Alta_Rentabilidade_Prev': alta_pred_cum,
    'Baixa_Vol_Prev': baixa_vol_pred_cum,
    'Baixa_Corr_Prev': baixa_corr_pred_cum
})



In [48]:
comparison_df.head()

,Alta_Rentabilidade,Baixa_Vol,Baixa_Corr,Mercado
Date,,,,
2023-12-04,0.999330,0.993387,0.990723,0.989852
2023-12-05,0.985691,0.999194,0.986069,0.990633
2023-12-06,0.975143,0.987992,0.978198,0.980641
2023-12-07,0.973784,0.994440,0.984465,0.983662
2023-12-08,0.976180,0.999877,0.986400,0.992124


In [49]:
# ================================================
# DataFrame com os Ativos de Cada Carteira
# ================================================
assets_df = pd.DataFrame({
    'Alta_Rentabilidade': top_assets.tolist(),
    'Baixa_Vol': low_vol_assets.tolist(),
    'Baixa_Corr': low_corr_assets.tolist()
})

In [50]:
assets_df

,Alta_Rentabilidade,Baixa_Vol,Baixa_Corr
0,AMBP3.SA,BBAS3.SA,EMBR3.SA
1,EMBR3.SA,ITUB4.SA,AMBP3.SA
2,MRFG3.SA,ABEV3.SA,PETR4.SA
3,BRFS3.SA,VALE3.SA,SYNE3.SA
4,WEGE3.SA,ELET3.SA,VALE3.SA


In [51]:
# Criação do gráfico Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=comparison_df.index,
    y=comparison_df['Alta_Rentabilidade'],
    mode='lines',
    name='Alta_Rentabilidade (Histórico)',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=comparison_df.index,
    y=comparison_df['Baixa_Vol'],
    mode='lines',
    name='Baixa_Vol (Histórico)',
    line=dict(color='orange')
))
fig.add_trace(go.Scatter(
    x=comparison_df.index,
    y=comparison_df['Baixa_Corr'],
    mode='lines',
    name='Baixa_Corr (Histórico)',
    line=dict(color='green')
))
fig.add_trace(go.Scatter(
    x=comparison_df.index,
    y=comparison_df['Mercado'],
    mode='lines',
    name='Mercado (Histórico)',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=predictions_df.index,
    y=predictions_df['Alta_Rentabilidade_Prev'],
    mode='lines+markers',
    line=dict(dash='dot', color='blue'),
    name='Alta_Rentabilidade (Previsão)'
))
fig.add_trace(go.Scatter(
    x=predictions_df.index,
    y=predictions_df['Baixa_Vol_Prev'],
    mode='lines+markers',
    line=dict(dash='dot', color='orange'),
    name='Baixa_Vol (Previsão)'
))
fig.add_trace(go.Scatter(
    x=predictions_df.index,
    y=predictions_df['Baixa_Corr_Prev'],
    mode='lines+markers',
    line=dict(dash='dot', color='green'),
    name='Baixa_Corr (Previsão)'
))

fig.update_layout(
    title="Comparação de Carteiras vs Mercado com Previsões Diárias para 60 Dias",
    xaxis_title="Data",
    yaxis_title="Crescimento Acumulado",
    legend_title="Séries",
    template="plotly_white"
)


In [52]:
# Salva o gráfico localmente como PNG
fig_filename = "comparacao_carteiras_mercado_plotly.png"
fig.write_image(fig_filename)

# Envia o gráfico PNG para o GCS
blob = bucket.blob(f"resultados/{fig_filename}")
blob.upload_from_filename(fig_filename)
print(f"Gráfico salvo no GCS: gs://{bucket_name}/resultados/{fig_filename}")

# Salvar gráfico interativo em HTML localmente
fig_html_filename = "comparacao_carteiras_mercado_plotly.html"
fig.write_html(fig_html_filename)

# Fazer upload do HTML para o GCS
html_blob = bucket.blob(f"resultados/{fig_html_filename}")
html_blob.upload_from_filename(fig_html_filename)
print(f"Gráfico interativo salvo no GCS: gs://{bucket_name}/resultados/{fig_html_filename}")

Gráfico salvo no GCS: gs://fiap-augusto/resultados/comparacao_carteiras_mercado_plotly.png
Gráfico interativo salvo no GCS: gs://fiap-augusto/resultados/comparacao_carteiras_mercado_plotly.html


In [53]:
# ================================================
# Salvar DataFrames no GCS
# ================================================
def save_df_to_gcs(df, gcs_path):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=True)
    blob = bucket.blob(gcs_path)
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')
    print(f"DataFrame salvo no GCS: gs://{bucket_name}/{gcs_path}")

save_df_to_gcs(predictions_df, "resultados/previsoes_60_dias.csv")

save_df_to_gcs(comparison_df, "resultados/comparison_df.csv")

# Salva também o DF de ativos de cada carteira
save_df_to_gcs(assets_df, "resultados/carteiras_ativos.csv")

DataFrame salvo no GCS: gs://fiap-augusto/resultados/previsoes_60_dias.csv
DataFrame salvo no GCS: gs://fiap-augusto/resultados/comparison_df.csv
DataFrame salvo no GCS: gs://fiap-augusto/resultados/carteiras_ativos.csv


In [54]:
# ================================================
# Carregar CSVs no BigQuery
# ================================================
client = bigquery.Client()

def load_csv_to_bq(uri, table_id, dataset_id=dataset_id):
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )
    load_job = client.load_table_from_uri(
        uri,
        f"{client.project}.{dataset_id}.{table_id}",
        job_config=job_config
    )
    load_job.result()
    print(f"Dados carregados no BigQuery na tabela {dataset_id}.{table_id} com sucesso!")

# Carrega as previsões para o BigQuery
load_csv_to_bq(f"gs://{bucket_name}/resultados/previsoes_60_dias.csv", "previsoes_carteiras")

# Carrega a tabela de ativos de cada carteira para o BigQuery
load_csv_to_bq(f"gs://{bucket_name}/resultados/carteiras_ativos.csv", "carteiras_ativos")

# Carrega as comparison_df para o BigQuery
load_csv_to_bq(f"gs://{bucket_name}/resultados/comparison_df.csv", "comparison_df")

print("Processo concluído.")

Dados carregados no BigQuery na tabela financeiro.previsoes_carteiras com sucesso!
Dados carregados no BigQuery na tabela financeiro.carteiras_ativos com sucesso!
Dados carregados no BigQuery na tabela financeiro.comparison_df com sucesso!
Processo concluído.
